In [1]:
import json
import csv
import pandas as pd
from sklearn import preprocessing

In [ ]:
### HELPER FUNCTIONS ###

# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_

In [2]:
#  tsv output file for cleaned up business json dataset
outfile = open("businesses.tsv", 'w')
businessfile = csv.writer(outfile, delimiter ="\t", quoting=csv.QUOTE_MINIMAL) 
businessfile.writerow(['business_id','name', 'stars', 'category'])

# Opens json dataset from path. 
with open('yelp_dataset/yelp_academic_dataset_business.json', encoding="utf-8") as f: 
    for line in f:
        row = json.loads(line)
        # Only getting businesses with review count over 20
        if row['review_count'] > 20:
            # some special char must be encoded in 'utf-8' 
            businessfile.writerow([row['business_id'], row['name'], row['stars'], row['categories']])

# Close tsv file
outfile.close()

# Create pandas dataframe tsv output file
df = pd.read_csv('businesses.tsv', delimiter ="\t", encoding="utf-8")

In [4]:
# Hashset for business_ids of businesses used for project
# Will use this to search for the businesses chosen for project in the reviews json dataset
# and add it to reviews tsv
chosen_business = {'YZeUH6zYS0dq5QHLYZhUnQ', 'oiAlXZPIFm2nBCt0DHLu_Q', 'fNil19SUfPAPnLQrYnFrGQ', 'JjcJVqhZXhP4tvOhg3fnag'}


#  tsv output file for cleaned up review json dataset
outfile = open("review_stars.tsv", 'w')
sfile = csv.writer(outfile, delimiter ="\t", quoting=csv.QUOTE_MINIMAL) 
sfile.writerow(['business_id','stars', 'text'])

# Opens json dataset from path. 
with open('yelp_dataset/yelp_academic_dataset_review.json', encoding="utf-8") as f: 
    for line in f:
        row = json.loads(line)
        # If statement to look for our chosen businesses
        if row['business_id'] in chosen_business: 
            # some special char must be encoded in 'utf-8' 
            sfile.writerow([row['business_id'], row['stars'], (row['text']).encode('utf-8')])

# Close tsv file
outfile.close()

# Create pandas dataframe tsv output file
df = pd.read_csv('review_stars.tsv', delimiter ="\t", encoding="utf-8")

print(df)

# One hot encoding for the 4 businesses chosen
encode_text_index(df,"business_id")

print(df)

                business_id  stars  \
0    oiAlXZPIFm2nBCt0DHLu_Q    5.0   
1    oiAlXZPIFm2nBCt0DHLu_Q    1.0   
2    oiAlXZPIFm2nBCt0DHLu_Q    5.0   
3    oiAlXZPIFm2nBCt0DHLu_Q    1.0   
4    oiAlXZPIFm2nBCt0DHLu_Q    5.0   
..                      ...    ...   
543  YZeUH6zYS0dq5QHLYZhUnQ    1.0   
544  YZeUH6zYS0dq5QHLYZhUnQ    1.0   
545  JjcJVqhZXhP4tvOhg3fnag    5.0   
546  JjcJVqhZXhP4tvOhg3fnag    1.0   
547  YZeUH6zYS0dq5QHLYZhUnQ    3.0   

                                                  text  
0    b"I've been coming to this dry cleaner for alm...  
1    b'They lost 2 pairs of my suitpants and told m...  
2    b"I have been going to this dry cleaning since...  
3    b'I\'ve only had my dry cleaning done here twi...  
4    b'After reading the reviews of the cleaners cl...  
..                                                 ...  
543  b'I hate to give a one star, but this place ne...  
544  b'I LOVE HOOTERS! When I came to this particul...  
545  b'Priced out purchasing o